In [1]:
print("hello world")

hello world


In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
documents = loader.load()
print(documents)

[Document(metadata={'source': 'speech.txt'}, page_content='The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to\ncontinue to be held as sacred and infallible? You must take a stand against the scriptures. Either you must stop the practice of\ncaste or you must stop believing in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It\nis a problem of overthrowing the authority of the shastras. So long as people believe in the sanctity of the shastras, they will\nnever be able to get rid of caste. The work of social reform is like the work of a gardener who is constantly pruning the leaves and\nbranches of a tree without ever attacking the roots. The real enemy is the belief in the shastras.\nWhat is your ideal society? My ideal society would be based on liberty, equality, and fraternity. I do not want to be the slave of\ntradition. I do not want to be the slave of others

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)



[Document(metadata={'source': 'speech.txt'}, page_content='The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to\ncontinue to be held as sacred and infallible? You must take a stand against the scriptures. Either you must stop the practice of\ncaste or you must stop believing in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It'), Document(metadata={'source': 'speech.txt'}, page_content='is a problem of overthrowing the authority of the shastras. So long as people believe in the sanctity of the shastras, they will\nnever be able to get rid of caste. The work of social reform is like the work of a gardener who is constantly pruning the leaves and\nbranches of a tree without ever attacking the roots. The real enemy is the belief in the shastras.\nWhat is your ideal society? My ideal society would be based on liberty, equality, and fraternity. I do not want to be the 

In [4]:
print(docs[0].page_content)

The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to
continue to be held as sacred and infallible? You must take a stand against the scriptures. Either you must stop the practice of
caste or you must stop believing in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
from langchain_chroma import Chroma

chromadb = Chroma.from_documents(documents=documents,
                                 embedding=embeddings)


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()
api = os.getenv("COHERE_API_KEY")

In [13]:
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-a-03-2025", cohere_api_key=api)
# result = model.invoke("what is your ideal society?")

In [14]:
llm.invoke("what is your ideal society?")

AIMessage(content='As an AI, I don\'t have personal preferences, emotions, or ideals, but I can describe a society that many people might consider "ideal" based on common values such as fairness, equality, sustainability, and well-being. Here are some key characteristics of such a society:\n\n1. **Equality and Justice**:  \n   - Everyone has equal access to opportunities, regardless of race, gender, socioeconomic status, or other factors.  \n   - Legal systems are fair, transparent, and prioritize justice and rehabilitation over punishment.  \n\n2. **Sustainability**:  \n   - The society prioritizes environmental stewardship, using renewable resources and minimizing waste.  \n   - Policies and practices are designed to combat climate change and preserve biodiversity.  \n\n3. **Education and Lifelong Learning**:  \n   - High-quality education is accessible to all, fostering critical thinking, creativity, and empathy.  \n   - Lifelong learning is encouraged, with resources available for 

In [9]:
from langchain.chains import RetrievalQA

retriever = chromadb.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [11]:
result = qa_chain.invoke({"query": "what is your ideal society?"})
print(result)

{'query': 'what is your ideal society?', 'result': 'My ideal society is one founded on the principles of **liberty, equality, and fraternity**. In such a society, individuals are free from the constraints of oppressive traditions and hierarchies, particularly those perpetuated by systems like caste. It is a society where everyone is treated as equal, regardless of birth or background, and where relationships are built on mutual respect and solidarity. This vision can only be realized by challenging and dismantling the authority of rigid, discriminatory scriptures (shastras) that uphold inequality. The goal is to create a world where no one is enslaved by tradition or by others, but instead lives in freedom, equality, and brotherhood.', 'source_documents': [Document(id='166da087-081a-411c-bc22-d4936a063330', metadata={'source': 'speech.txt'}, page_content='The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to\